# Multi-Agent Customer Service System with A2A and MCP

This notebook demonstrates a multi-agent customer service system using:
- **LangGraph** for agent orchestration
- **Claude Sonnet 4** for agent reasoning
- **MCP Protocol** for tool integration
- **A2A Communication** for agent coordination

## Architecture
- Router Agent (Orchestrator)
- Customer Data Agent (Specialist)
- Support Agent (Specialist)

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q anthropic langchain langchain-anthropic langchain-core langgraph typing-extensions

In [ ]:
# Import required libraries
import os
import sqlite3
import json
from datetime import datetime
from typing import TypedDict, Annotated, List, Dict, Any, Optional
import operator
import random

from langgraph.graph import StateGraph, END
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

print("✅ All imports successful!")

In [ ]:
# Set up API key
from google.colab import userdata

# Store your API key in Colab secrets as 'ANTHROPIC_API_KEY'
try:
    os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Fallback: manual entry
    api_key = input("Enter your Anthropic API key: ")
    os.environ['ANTHROPIC_API_KEY'] = api_key
    print("✅ API key set manually")

## 2. Database Setup

In [ ]:
# Initialize database with sample data
def setup_database(db_path='support.db'):
    """Initialize database with schema and test data"""
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Drop existing tables
    cursor.execute('DROP TABLE IF EXISTS tickets')
    cursor.execute('DROP TABLE IF EXISTS customers')
    
    # Create customers table
    cursor.execute('''
        CREATE TABLE customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT,
            phone TEXT,
            status TEXT DEFAULT 'active',
            tier TEXT DEFAULT 'standard',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Create tickets table
    cursor.execute('''
        CREATE TABLE tickets (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_id INTEGER NOT NULL,
            issue TEXT NOT NULL,
            status TEXT DEFAULT 'open',
            priority TEXT DEFAULT 'medium',
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (customer_id) REFERENCES customers (id)
        )
    ''')
    
    # Insert test customers
    customers = [
        ('Alice Johnson', 'alice@email.com', '555-0101', 'active', 'premium'),
        ('Bob Smith', 'bob@email.com', '555-0102', 'active', 'standard'),
        ('Charlie Davis', 'charlie@email.com', '555-0103', 'active', 'enterprise'),
        ('Diana Wilson', 'diana@email.com', '555-0104', 'disabled', 'standard'),
        ('Eve Martinez', 'eve@email.com', '555-0105', 'active', 'premium'),
    ]
    
    cursor.executemany('''
        INSERT INTO customers (name, email, phone, status, tier)
        VALUES (?, ?, ?, ?, ?)
    ''', customers)
    
    # Insert test tickets
    tickets = [
        (1, "Unable to login to account", "open", "high"),
        (1, "Billing discrepancy", "resolved", "medium"),
        (2, "Feature request", "open", "low"),
        (3, "API access issues", "in_progress", "high"),
        (5, "Account upgrade inquiry", "open", "medium"),
    ]
    
    cursor.executemany('''
        INSERT INTO tickets (customer_id, issue, status, priority)
        VALUES (?, ?, ?, ?)
    ''', tickets)
    
    conn.commit()
    
    cursor.execute('SELECT COUNT(*) FROM customers')
    customer_count = cursor.fetchone()[0]
    
    cursor.execute('SELECT COUNT(*) FROM tickets')
    ticket_count = cursor.fetchone()[0]
    
    conn.close()
    
    print(f"✅ Database initialized!")
    print(f"   - {customer_count} customers")
    print(f"   - {ticket_count} tickets")
    
    return db_path

# Run database setup
DB_PATH = setup_database()

## 3. MCP Server Implementation

In [ ]:
# MCP Server with all tools

def get_db_connection():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

# MCP Tool: Get Customer
def get_customer(customer_id: int) -> Dict[str, Any]:
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()
        return dict(row) if row else {"error": f"Customer {customer_id} not found"}
    except Exception as e:
        return {"error": str(e)}

# MCP Tool: List Customers
def list_customers(status: Optional[str] = None, limit: int = 10) -> List[Dict[str, Any]]:
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        if status:
            cursor.execute('SELECT * FROM customers WHERE status = ? LIMIT ?', (status, limit))
        else:
            cursor.execute('SELECT * FROM customers LIMIT ?', (limit,))
        rows = cursor.fetchall()
        conn.close()
        return [dict(row) for row in rows]
    except Exception as e:
        return [{"error": str(e)}]

# MCP Tool: Get Customer History
def get_customer_history(customer_id: int) -> Dict[str, Any]:
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        customer_row = cursor.fetchone()
        if not customer_row:
            conn.close()
            return {"error": f"Customer {customer_id} not found"}
        
        customer = dict(customer_row)
        
        cursor.execute('SELECT * FROM tickets WHERE customer_id = ? ORDER BY created_at DESC', (customer_id,))
        ticket_rows = cursor.fetchall()
        conn.close()
        
        tickets = [dict(row) for row in ticket_rows]
        
        return {
            "customer": customer,
            "tickets": tickets,
            "total_tickets": len(tickets)
        }
    except Exception as e:
        return {"error": str(e)}

# MCP Tool: Create Ticket
def create_ticket(customer_id: int, issue: str, priority: str = 'medium') -> Dict[str, Any]:
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        cursor.execute('SELECT id FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {"error": f"Customer {customer_id} not found"}
        
        cursor.execute('''
            INSERT INTO tickets (customer_id, issue, status, priority)
            VALUES (?, ?, 'open', ?)
        ''', (customer_id, issue, priority))
        
        ticket_id = cursor.lastrowid
        conn.commit()
        
        cursor.execute('SELECT * FROM tickets WHERE id = ?', (ticket_id,))
        row = cursor.fetchone()
        conn.close()
        
        return dict(row)
    except Exception as e:
        return {"error": str(e)}

# MCP Server Class
class MCPServer:
    def __init__(self):
        self.tools = {
            'get_customer': get_customer,
            'list_customers': list_customers,
            'get_customer_history': get_customer_history,
            'create_ticket': create_ticket
        }
    
    def call_tool(self, tool_name: str, **kwargs) -> Any:
        if tool_name not in self.tools:
            return {"error": f"Tool {tool_name} not found"}
        try:
            return self.tools[tool_name](**kwargs)
        except Exception as e:
            return {"error": f"Tool execution failed: {str(e)}"}
    
    def list_tools(self) -> List[str]:
        return list(self.tools.keys())

# Initialize MCP server
mcp_server = MCPServer()
print(f"✅ MCP Server initialized with tools: {mcp_server.list_tools()}")

## 4. Multi-Agent System Implementation

In [ ]:
# State definition
class AgentState(TypedDict):
    query: str
    messages: Annotated[List[Dict[str, Any]], operator.add]
    current_agent: str
    next_agent: Optional[str]
    customer_data: Optional[Dict[str, Any]]
    tickets: Optional[List[Dict[str, Any]]]
    final_response: str
    coordination_log: Annotated[List[str], operator.add]
    task_type: Optional[str]

# Initialize LLM
llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)

print("✅ State and LLM initialized")

In [ ]:
# Router Agent
class RouterAgent:
    def __init__(self):
        self.name = "Router"
        
    def analyze_query(self, query: str) -> Dict[str, Any]:
        system_prompt = """You are a Router Agent. Analyze queries and return JSON:
        {
            "task_type": "data_retrieval|support|complex",
            "agents_needed": ["customer_data", "support"],
            "priority": "low|medium|high"
        }"""
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"Analyze: {query}")
        ]
        
        response = llm.invoke(messages)
        
        try:
            return json.loads(response.content)
        except:
            return {
                "task_type": "support",
                "agents_needed": ["support"],
                "priority": "medium"
            }
    
    def __call__(self, state: AgentState) -> AgentState:
        query = state['query']
        state['coordination_log'].append(f"[ROUTER] Received: {query}")
        
        analysis = self.analyze_query(query)
        state['task_type'] = analysis['task_type']
        
        agents_needed = analysis['agents_needed']
        state['next_agent'] = agents_needed[0] if agents_needed else 'support'
        
        state['coordination_log'].append(f"[ROUTER] Routing to {state['next_agent']}")
        state['current_agent'] = 'router'
        
        state['messages'].append({
            "from": "router",
            "to": state['next_agent'],
            "content": query
        })
        
        return state

print("✅ Router Agent defined")

In [ ]:
# Customer Data Agent
class CustomerDataAgent:
    def __init__(self):
        self.name = "CustomerData"
        
    def extract_customer_id(self, text: str) -> Optional[int]:
        import re
        patterns = [r'customer\s+id\s+(\d+)', r'id\s+(\d+)', r'customer\s+(\d+)']
        for pattern in patterns:
            match = re.search(pattern, text.lower())
            if match:
                return int(match.group(1))
        return None
    
    def __call__(self, state: AgentState) -> AgentState:
        query = state['query']
        state['coordination_log'].append("[DATA] Processing request")
        
        customer_id = self.extract_customer_id(query)
        response_data = {}
        
        if customer_id:
            customer_info = mcp_server.call_tool('get_customer', customer_id=customer_id)
            state['customer_data'] = customer_info
            response_data['customer'] = customer_info
            
            state['coordination_log'].append(f"[DATA] Retrieved customer {customer_id}")
            
            if 'ticket' in query.lower() or 'history' in query.lower():
                history = mcp_server.call_tool('get_customer_history', customer_id=customer_id)
                state['tickets'] = history.get('tickets', [])
                response_data['history'] = history
                state['coordination_log'].append(f"[DATA] Retrieved {len(state['tickets'])} tickets")
        
        state['current_agent'] = 'customer_data'
        state['next_agent'] = 'support'
        state['coordination_log'].append("[DATA] Forwarding to Support")
        
        state['messages'].append({
            "from": "customer_data",
            "to": "support",
            "data": response_data
        })
        
        return state

print("✅ Customer Data Agent defined")

In [ ]:
# Support Agent
class SupportAgent:
    def __init__(self):
        self.name = "Support"
        
    def generate_response(self, state: AgentState) -> str:
        query = state['query']
        customer_data = state.get('customer_data')
        tickets = state.get('tickets', [])
        
        context = f"Query: {query}\n"
        if customer_data:
            context += f"Customer: {json.dumps(customer_data, indent=2)}\n"
        if tickets:
            context += f"Tickets: {len(tickets)}\n"
            for t in tickets[:3]:
                context += f"  - {t.get('issue')} [{t.get('status')}]\n"
        
        system_prompt = """You are a Support Agent. Provide helpful, professional
        responses using available customer data. Be concise and actionable."""
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=context)
        ]
        
        response = llm.invoke(messages)
        return response.content
    
    def __call__(self, state: AgentState) -> AgentState:
        state['coordination_log'].append("[SUPPORT] Generating response")
        
        response = self.generate_response(state)
        state['final_response'] = response
        state['current_agent'] = 'support'
        state['next_agent'] = 'final'
        
        state['coordination_log'].append("[SUPPORT] Response complete")
        
        return state

print("✅ Support Agent defined")

In [ ]:
# Create workflow
def create_workflow():
    workflow = StateGraph(AgentState)
    
    router = RouterAgent()
    customer_data = CustomerDataAgent()
    support = SupportAgent()
    
    workflow.add_node("router", router)
    workflow.add_node("customer_data", customer_data)
    workflow.add_node("support", support)
    
    def route_after_router(state):
        next_agent = state.get('next_agent', 'support')
        return END if next_agent == 'final' else next_agent
    
    def route_after_data(state):
        return 'support'
    
    def route_after_support(state):
        return END
    
    workflow.set_entry_point("router")
    workflow.add_conditional_edges("router", route_after_router,
                                   {"customer_data": "customer_data", "support": "support", END: END})
    workflow.add_conditional_edges("customer_data", route_after_data,
                                   {"support": "support"})
    workflow.add_conditional_edges("support", route_after_support,
                                   {END: END})
    
    return workflow.compile()

print("✅ Workflow created")

In [ ]:
# Main process function
def process_query(query: str, verbose: bool = True):
    initial_state = {
        "query": query,
        "messages": [],
        "current_agent": None,
        "next_agent": None,
        "customer_data": None,
        "tickets": None,
        "final_response": "",
        "coordination_log": [],
        "task_type": None
    }
    
    app = create_workflow()
    final_state = app.invoke(initial_state)
    
    if verbose:
        print("\n" + "="*80)
        print("COORDINATION LOG")
        print("="*80)
        for log in final_state['coordination_log']:
            print(log)
        print("="*80)
    
    return final_state

print("✅ Process function ready")

## 5. Test Scenarios

In [ ]:
# Test 1: Simple Query
print("\n" + "="*80)
print("TEST 1: Simple Query")
print("="*80)

result = process_query("Get customer information for ID 1")
print(f"\nFINAL RESPONSE:\n{result['final_response']}")

In [ ]:
# Test 2: Coordinated Query
print("\n" + "="*80)
print("TEST 2: Coordinated Query")
print("="*80)

result = process_query("I'm customer 1 and need help upgrading my account")
print(f"\nFINAL RESPONSE:\n{result['final_response']}")

In [ ]:
# Test 3: Complex Query with History
print("\n" + "="*80)
print("TEST 3: Complex Query with Ticket History")
print("="*80)

result = process_query("Show me the ticket history for customer 3")
print(f"\nFINAL RESPONSE:\n{result['final_response']}")

In [ ]:
# Test 4: Escalation
print("\n" + "="*80)
print("TEST 4: Escalation (Urgent Issue)")
print("="*80)

result = process_query("I've been charged twice! This is urgent. Customer ID 2")
print(f"\nFINAL RESPONSE:\n{result['final_response']}")

## 6. Custom Query Testing

In [ ]:
# Test your own query
custom_query = "Show me all premium customers"  # Modify this

result = process_query(custom_query)
print(f"\nFINAL RESPONSE:\n{result['final_response']}")

## Summary

This notebook demonstrates:
- ✅ Multi-agent orchestration with LangGraph
- ✅ A2A communication and coordination
- ✅ MCP protocol for tool integration
- ✅ State management across agents
- ✅ Dynamic routing based on query analysis
- ✅ Real-world customer service scenarios